In [1]:
import pandas as pd
import re
import numpy as np

In [12]:
train = pd.read_csv('train.csv',nrows=35000000,dtype={'Semana':int,'Agencia_ID':int,
                                       'Canal_ID':int,'Ruta_SAK':int,
                                      'Cliente_ID':int,'Producto_ID':int,
                                      'Venta_uni_hoy':int,'Venta_hoy':float,
                                      'Dev_uni_proxima':int,'Dev_proxima':float,
                                      'Demanda_uni_equil':int})

train.columns =['Week_number','Sales_Depot_ID','Sales_Channel_ID','Route_ID','Client_ID','Product_ID',
                                     'Sales_unit_this_week','Sales_this_week', 'Returns_unit_next_week','Returns_next_week', 
                                     'Adjusted_Demand']
train.loc[:,'Sales_this_week']*=0.056
train.loc[:,'Returns_next_week']*=0.056

'''
train1 = pd.read_csv('train.csv',skiprows=range(1, 35000001),dtype={'Semana':int,'Agencia_ID':int,
                                       'Canal_ID':int,'Ruta_SAK':int,
                                      'Cliente_ID':int,'Producto_ID':int,
                                      'Venta_uni_hoy':int,'Venta_hoy':float,
                                      'Dev_uni_proxima':int,'Dev_proxima':float,
                                      'Demanda_uni_equil':int})

train1.columns =['Week_number','Sales_Depot_ID','Sales_Channel_ID','Route_ID','Client_ID','Product_ID',
                                     'Sales_unit_this_week','Sales_this_week', 'Returns_unit_next_week','Returns_next_week', 
                                     'Adjusted_Demand']
train1.loc[:,'Sales_this_week']*=0.056
train1.loc[:,'Returns_next_week']*=0.056
'''

train.head()

,Week_number,Sales_Depot_ID,Sales_Channel_ID,Route_ID,Client_ID,Product_ID,Sales_unit_this_week,Sales_this_week,Returns_unit_next_week,Returns_next_week,Adjusted_Demand
0,3,1110,7,3301,15766,1212,3,1.40784,0,0.0,3
1,3,1110,7,3301,15766,1216,4,1.87712,0,0.0,4
2,3,1110,7,3301,15766,1238,4,2.20192,0,0.0,4
3,3,1110,7,3301,15766,1240,4,1.87712,0,0.0,4
4,3,1110,7,3301,15766,1242,3,1.28352,0,0.0,3


In [10]:
test  = pd.read_csv('test.csv')
client = pd.read_csv('client.csv')
product = pd.read_csv('product.csv')
town = pd.read_csv('town_state.csv')

test.columns = ['id', 'Week_number', 'Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID','Product_ID']
client.columns = ['Client_ID', 'Client_name']
product.columns = ['Product_ID', 'Product_Name']
town.columns = ['Sales_Depot_ID', 'Town', 'State']

#in client table some words mean not defiened

client.loc[(client['Client_name'] == 'NO IDENTIFICADO') | (client['Client_name'] == 'SIN NOMBRE') |
           (client['Client_name'] == 'INACTIVAR NO TOCAR') ,'Client_name' ] = 'NO_Name'

In [11]:
product_id_list = product['Product_ID'].values
product_name_list = product['Product_Name'].values
num_product = product_name_list.shape[0]

product_weight_list = np.empty(num_product)
product_weight_list[:] = np.NAN
product_piece_list = np.ones(num_product)
product_brand_list = list()
product_short_name_list = list()

product_info_list = zip(product_id_list,product_name_list)

#extract weight,piece,brand and prod_name

regex_piece_weight = re.compile(r'(.*?)(\d+)(p|P)(?!ct).*?(\d+)\s*(kg|Kg|g|G|ml)') #check for line with piece first and then weight
regex_weight_piece = re.compile(r'(.*?)(\d+)\s*(kg|Kg|g|G|ml).*?(\d+)(p|P)(?!ct|CT)') #opp of first
regex_weight = re.compile(r'(.*?)(\d+)\s*(kg|Kg|g|G|ml)')  #line with only weights
regex_piece = re.compile(r'(.*?)(\d+)(p|P)(?!ct|CT)')  #line with only piece

dont_know_list = ['MTB', 'MTA', 'TNB', 'CU', 'TAB', 'Prom']

for i, (product_id, orig_product_name) in enumerate(product_info_list):
    product_name = orig_product_name.rstrip(str(product_id)).strip()
    
    product_name_words = product_name.split()   
    
    if re.search(regex_piece_weight, product_name):
        regex_result = re.search(regex_piece_weight, product_name)
        product_weight_list[i] = float(regex_result.group(4))
        product_piece_list[i] = int(regex_result.group(2))
        product_short_name = regex_result.group(1)
        if regex_result.group(5) == 'kg' or regex_result.group(5) == 'Kg':
            product_weight_list[i] *= 1000
            
    elif re.search(regex_weight_piece, product_name):
        regex_result = re.search(regex_weight_piece, product_name)
        product_weight_list[i] = float(regex_result.group(2))
        product_piece_list[i] = int(regex_result.group(4))
        if regex_result.group(3) == 'kg' or regex_result.group(3) == 'Kg':
            product_weight_list[i] *= 1000
        product_short_name = regex_result.group(1)
        
    elif re.search(regex_weight, product_name):
        regex_result = re.search(regex_weight, product_name)
        product_weight_list[i] = float(regex_result.group(2))
        if regex_result.group(3) == 'kg' or regex_result.group(3) == 'Kg':
            product_weight_list[i] *= 1000
        product_short_name = regex_result.group(1)
        
    elif re.search(regex_piece, product_name):
        regex_result = re.search(regex_piece, product_name)
        product_piece_list[i] = int(regex_result.group(2))
        product_short_name = regex_result.group(1) 
        
    else:
        product_short_name = ' '.join([word for word in product_name_words[:-1] if word not in dont_know_list])
        
    product_brand_list.append(product_name_words[-1])
    product_short_name_list.append(product_short_name)
    
product['product_name'] = product_short_name_list
product['brand'] = product_brand_list
product['weight'] = product_weight_list
product['pieces'] = product_piece_list

product.drop('Product_Name',axis = 1,inplace=True)
product.head()

,Product_ID,product_name,brand,weight,pieces
0,0,NO,IDENTIFICADO,NaN,1.0
1,9,Capuccino Moka,NES,750.0,1.0
2,41,Bimbollos Ext sAjonjoli,BIM,480.0,6.0
3,53,Burritos Sincro,LON,170.0,1.0
4,72,Div Tira Mini Doradita,TR,45.0,4.0


In [12]:
test1= test
test1.set_index('Product_ID',inplace=True)
product.set_index('Product_ID',inplace=True)
t = pd.concat([test1, product], axis=1, join='inner')
t.reset_index(inplace=True)
t.shape

(6999251, 11)

In [13]:
t.set_index('Sales_Depot_ID',inplace=True)
town.set_index('Sales_Depot_ID',inplace=True)
t = pd.concat([t, town], axis=1, join='inner')
t.reset_index(inplace=True)
t.shape

(6999251, 13)

In [14]:
t.head()

,Sales_Depot_ID,Product_ID,id,Week_number,Sales_Channel_ID,Route_ID,Client_ID,product_name,brand,weight,pieces,Town,State
0,4037,35305,0,11,1,2209,4639078,Barritas Fresa,MLA,75.0,1.0,2499 REYNOSA MARINELA,TAMAULIPAS
1,2237,1238,1,11,1,1226,4705135,Panquecito Gota Choc,BIM,140.0,2.0,2402 TUXTLA,CHIAPAS
2,2045,32940,2,10,1,2831,4549769,Principe Choc Bco,MLA,110.0,10.0,2163 LA PIEDAD,MICHOACÁN
3,1227,43066,3,11,1,4448,4717855,Pay Pina,MLA,85.0,1.0,2066 TOLUCA PONIENTE,ESTADO DE MÉXICO
4,1219,1277,4,11,1,1130,966351,Nito Edicion Especial,BIM,62.0,1.0,2042 AG. TEPOZOTLAN,ESTADO DE MÉXICO
